In [2]:
import pandas as pd
import numpy as np
import altair as alt

In [8]:
from sklearn import preprocessing, decomposition, model_selection, metrics, pipeline
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.cluster import kmeans_plusplus
from nltk.corpus import stopwords
import nltk
import re

In [9]:
nltk.download("stopwords")

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\gigi\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [12]:
letras = pd.read_csv("letras-ptbr-samba-grande.csv")
letras.head()

,SName,Lyric,Artist,Songs,Popularity,Genre,Genres
0,1967,Nascido em São Cristóvão. Morador de Madureira...,Marcelo D2,115,4.6,Samba,Hip Hop; Rap; Samba; Rock; Black Music; Reggae...
1,Febre do Rato,Não beijo pé do patrão. Não quero o que é dos ...,Marcelo D2,115,4.6,Samba,Hip Hop; Rap; Samba; Rock; Black Music; Reggae...
2,À Procura da Batida Perfeita,À procura da batida perfeita. Então corre a ba...,Marcelo D2,115,4.6,Samba,Hip Hop; Rap; Samba; Rock; Black Music; Reggae...
3,Desabafo,"[Introdução]. Deixa,deixa,deixa. Eu dizer o qu...",Marcelo D2,115,4.6,Samba,Hip Hop; Rap; Samba; Rock; Black Music; Reggae...
4,Qual É?,"Ih, eu tenho algo a dizer. Explicar pra você. ...",Marcelo D2,115,4.6,Samba,Hip Hop; Rap; Samba; Rock; Black Music; Reggae...


In [13]:
letras.sample(10)

,SName,Lyric,Artist,Songs,Popularity,Genre,Genres
760,Pra Que Negar,pra que amor. se não veio de dentro. então pra...,Alcione,374,5.1,Samba,Samba; Romântico; MPB; Pagode; Axé; Bossa Nova...
6523,13 de Dezembro,O tempo passou. Já chegou ao fim. É a despedid...,Mumuzinho,71,4.3,Samba,Pagode; Romântico; Samba; MPB; Sertanejo; Gosp...
3486,Chuvas de Verão,Podemos ser amigos simplesmente. Coisas do amo...,Elza Soares,148,1.5,Samba,Samba; MPB; Jazz; Bossa Nova; Soul Music; Blac...
585,Delírios de Amor,Você acaba dando um jeito de me dominar. Eu nã...,Alcione,374,5.1,Samba,Samba; Romântico; MPB; Pagode; Axé; Bossa Nova...
6450,Quando Essa Onda Passar,Quando essa onda passar. Vou te levar nas fave...,Martinho da Vila,217,5.6,Samba,Samba; Samba Enredo; Axé; Forró; Bossa Nova; I...
8028,"Pot-Pourri ""Raça Negra 2"": De Bem Com A Vida /...",Aí meu camarada vê se fica na sua. porque o me...,Só Pra Contrariar,157,4.7,Samba,Samba; Pagode; Romântico; Gospel/Religioso; Jo...
7233,Deixa Pra Mim,Senti pela primeira vez. A força de uma paixão...,Raça Negra,280,14.3,Samba,Romântico; Samba; Pagode; Axé; MPB; Regional; ...
7321,Navegantes da Paixão,Estou na cauda de um cometa. Vôo em nuvens de ...,Raça Negra,280,14.3,Samba,Romântico; Samba; Pagode; Axé; MPB; Regional; ...
8184,Simples Desejo (Part. Gilberto Gil),Que tal abrir a porta do dia. Entrar sem pedir...,Thiaguinho,143,13.8,Samba,Pagode; Romântico; Samba; Trilha Sonora; Black...
2190,Você Não Gosta de Você,"Certo é que você já percebeu,. que eu sempre q...",Bezerra da Silva,292,2.9,Samba,Pagode; Samba; MPB; Instrumental; Axé; Heavy M...


In [14]:
letras.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 8662 entries, 0 to 8661
Data columns (total 7 columns):
 #   Column      Non-Null Count  Dtype  
---  ------      --------------  -----  
 0   SName       8662 non-null   object 
 1   Lyric       8662 non-null   object 
 2   Artist      8662 non-null   object 
 3   Songs       8662 non-null   int64  
 4   Popularity  8662 non-null   float64
 5   Genre       8662 non-null   object 
 6   Genres      8662 non-null   object 
dtypes: float64(1), int64(1), object(5)
memory usage: 473.8+ KB


In [15]:
letras["Genre"].value_counts()

Samba    8662
Name: Genre, dtype: int64

In [17]:
letras["Artist"].value_counts()[0:20]

Alcione              372
Chico Buarque        361
Fundo de Quintal     325
Bezerra da Silva     303
Zeca Pagodinho       282
Raça Negra           260
Jorge Ben Jor        259
Maria Rita           248
Marcelo D2           248
Belo                 238
ExaltaSamba          232
Martinho da Vila     231
Harmonia do Samba    218
Arlindo Cruz         196
Clara Nunes          192
Beth Carvalho        190
Turma do Pagode      174
Pixote               172
Jorge Aragão         170
Alexandre Pires      168
Name: Artist, dtype: int64

In [18]:
stop_words = set(stopwords.words("portuguese"))
print(len(stop_words))

207


In [23]:
clean_lyrics = []
for w in range(len(letras.Lyric)):
    lyric = letras["Lyric"][w]

    # remove special characteres and digits
    lyric = re.sub("(\\d|\\W)+|\w*\d\w*", " ", lyric)
    lyric = ' '.join(s for s in lyric.split() if (not any(c.isdigit() for c in s)) and len(s) > 2)
    clean_lyrics.append(lyric)

clean_lyrics[0:5]

['Nascido São Cristóvão Morador Madureira Desde pequeno acostumado subir ladeira lembro muito bem dos meus tempos moleque que sempre passava férias final Padre Miguel sempre bateria saudoso Mestre André sempre soube que queria futebol rua campo baixo Você sabe Meu tio gentil era esculacho andava pelas ruas vestindo meu bate bola passasse minha frente Era melhor sair fora Carnaval rua perigoso divertido Mas passei por tudo isso Entre mortos feridos Graças meu pai pessoal tramela Sérgio Cabrito meu padrinho Não dava trégua Lembra Cassino Bangu vez quando Curtir funk ver mulherada rebolar Kool and the gang gap band outro mestre James Brown Era alegria Não tinha pau quero ver homem mané jeito que fui que sou quero ver homem mané Que nem parteira falou Andaraí Grajaú bicho pegava mais Quando pichava muro Sempre tinha correndo atrás Carlos Peixe meu camarada vez quando piche Outras baforada Vida moleque sempre sangue bom Calote ônibus Pra praia verão Pra ficar pouco mais Roubava supermercado

# Vetores TF-IDF

In [27]:
# TF - IDF vectorizer
tfv = TfidfVectorizer(
    min_df = 5,
    max_df = 0.9,
    max_features = None,
    stop_words = stop_words,
    ngram_range = (1, 3)
)

# transform
vec_text = tfv.fit_transform(clean_lyrics)

words = tfv.get_feature_names()

len(words)

c:\Users\gigi\AppData\Local\Programs\Python\Python310\lib\site-packages\sklearn\utils\deprecation.py:87: FutureWarning: Function get_feature_names is deprecated; get_feature_names is deprecated in 1.0 and will be removed in 1.2. Please use get_feature_names_out instead.
  warnings.warn(msg, category=FutureWarning)


23874

In [26]:
words[1:10]

['abacaxi',
 'abaeté',
 'abafar',
 'abaixa',
 'abaixar',
 'abaixar cabeça',
 'abaixo',
 'abaixo deus',
 'abaixou']

Agrupamento

Decidindo K

In [29]:
from sklearn.cluster import MiniBatchKMeans


qualidade = pd.DataFrame(columns = ['k', 'ssd'])
for k in range(1, 17, 1):
    # kemans = KMeans(n_clusters=k, rantom_state=0)
    kmeans = MiniBatchKMeans(n_clusters=k, init_size=1024, batch_size=2048, random_state=20)
    kmeans.fit(vec_text)
    qualidade = qualidade.append({'k': k, 'ssd': kmeans.inertia_}, ignore_index=True)

alt.Chart(qualidade).mark_line(
    point=True
).encode(
    x = 'k',
    y = alt.Y('ssd', scale = alt.Scale(zero=False))
)




C:\Users\gigi\AppData\Local\Temp\ipykernel_8264\3478104648.py:8: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  qualidade = qualidade.append({'k': k, 'ssd': kmeans.inertia_}, ignore_index=True)
C:\Users\gigi\AppData\Local\Temp\ipykernel_8264\3478104648.py:8: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  qualidade = qualidade.append({'k': k, 'ssd': kmeans.inertia_}, ignore_index=True)
C:\Users\gigi\AppData\Local\Temp\ipykernel_8264\3478104648.py:8: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  qualidade = qualidade.append({'k': k, 'ssd': kmeans.inertia_}, ignore_index=True)
C:\Users\gigi\AppData\Local\Temp\ipykernel_8264\3478104648.py:8: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a

alt.Chart(...)

Agrupando com o K escolhido

In [31]:
# setup kmeans clustering
kmeans = MiniBatchKMeans(n_clusters=7, init_size=1024, batch_size=2048, random_state=20)

# fit the data
kmeans.fit(vec_text)
labels = kmeans.predict(vec_text)

letras_ag = letras.assign(grupo = labels)

In [32]:
letras_ag.sample(10)

,SName,Lyric,Artist,Songs,Popularity,Genre,Genres,grupo
3732,Medley - Como Nunca Amei Ninguem / Me Apaixone...,Canta: Jeffinho. Nunca pensei que você. Me dei...,ExaltaSamba,238,13.0,Samba,Samba,0
8613,Sambou.... Sambou,Sambou sambou não descansou. Ficou zangada qua...,Zeca Pagodinho,309,13.2,Samba,Samba; Pagode; Bossa Nova; MPB; Jovem Guarda; ...,6
1460,Varal,O samba é a razão da minha vida. O samba me fe...,Art Popular,184,3.2,Samba,Pagode; Samba; Romântico; Dance; Pop; Hip Hop;...,6
589,Devolvi,Devolvi. O cordão e a medalha de ouro. E tudo ...,Alcione,374,5.1,Samba,Samba; Romântico; MPB; Pagode; Axé; Bossa Nova...,2
6720,Amor é Assim,Noto pelo seu procedimento. Que o seu coração....,Paulinho da Viola,160,0.7,Samba,Samba; MPB; Instrumental; Blues; Sertanejo; Ax...,4
6911,Linda Voz (Olá),Olá.... Hoje eu te vi pela televisão. Mostrand...,Péricles,102,6.8,Samba,Romântico; Pagode; Samba; Sertanejo; Samba Enr...,1
1493,À Meia Luz,"Jantar com ela, à luz de vela,. É tudo que eu ...",Belo,236,14.7,Samba,Romântico; Pagode; Samba; Trilha Sonora; Axé; ...,2
4369,Vê Se Me Escuta,Lêlelele laia laia laia laia laia laia. Laia l...,Grupo Revelação,188,13.0,Samba,Pagode; Samba; Romântico; Sertanejo; Samba Enr...,1
4391,Jura de Amor,Jura de Amor. Você veio de repende e reacendeu...,Harmonia do Samba,221,2.1,Samba,Axé; Samba; Pagode; Romântico; Electro Swing; ...,4
7115,Pedindo Pra Ficar,"Mais uma Vez a gente pode ver,. como dói a dor...",Pixote,167,8.8,Samba,Romântico; Samba; Pagode; Trilha Sonora; Funk;...,2


Interpretando os grupos

In [33]:
letras_ag['grupo'].value_counts()

1    3102
4    1707
2    1469
3     925
0     867
6     534
5      58
Name: grupo, dtype: int64

In [35]:
# this loop transform the numbers back into words
common_words = kmeans.cluster_centers_.argsort()[:,-1:-15:-1]
for num, centroid in enumerate(common_words):
    print(str(num) + ' : ' + ', '.join(words[word] for word in centroid))

0 : sei, amo, amor, pra, mim, quero, coração, tudo, assim, bem, sempre, agora, feliz, vida
1 : vai, pra, gente, deus, vem, mulher, dia, bem, vida, mar, quer, todo, faz, ver
2 : pra, tudo, mim, coração, amor, dia, gente, mundo, vai, vida, vem, todo, tempo, bem
3 : vou, pra, vai, amor, mim, quero, sei, nada, fazer, bem, vida, tudo, ver, agora
4 : amor, pra, quero, amar, coração, vem, mim, assim, faz, tão, bem, vai, paixão, gente
5 : laiá, laiá laiá, laiá laiá laiá, malandro, lugar, lalaiá, pra, lalaiá laiá, oba, amor, laiá lalaiá, vai, mundo, pode
6 : samba, sambar, vai, pra, vem, roda, povo, vou, gente, samba samba, cantar, ver, samba vai, palma


Quais são algumas m[usicas do grupo X?

In [36]:
pd.options.display.max_colwidth = 100
letras_ag.query('grupo == 5')[["SName", "Lyric", "Artist"]].sample(10)

,SName,Lyric,Artist
7799,Sobe na Cadeira (Part. Thiaguinho),"Sobe na cadeira aê, vai, vai, vai. Sobe na cadeira aê, vai, vai, vai. Sobe na cadeira aê, vai, v...",Samprazer
4134,Verdadeira Chama,"E digue, digue, di deo oba. E digue, digue, di deo oba. E digue, digue, di deo oba. Lê lê lê lê ...",Fundo de Quintal
94,Pode Acreditar (Meu Laiá Laiá) (part. Seu Jorge),"Laiá Laiá, chega como eu cheguei. Laiá Laiá Laiá, malandro é malandro. Laiá Laiá, pisa como eu p...",Marcelo D2
3212,Tempos Difíceis,Tempos difíceis pra quem sonha com justiça. Pra quem quer dignidade em cada palmo desse chão. Te...,Diogo Nogueira
1211,Meu Laia-raiá,Você é meu povo. Você é meu samba. Você é a bossa. E a minha voz. Prá você eu trago. Um sambinha...,Arlindo Cruz
8522,Martim Cererê,Lá lá lá lá lauê. Fala Martim Cererê. Lá lá lá lá lauê. Fala Martim Cererê. Vem cá Brasil. Deixa...,Zeca Pagodinho
1212,Meu Lugar,"O meu lugar,. é caminho de Ogum e Iansã,. lá tem samba até de manhã,. uma ginga em cada andar.. ...",Arlindo Cruz
6952,Viola Em Bandoleira/Sorriso Aberto,"Andei, andei. Andei, andei. De violão e bandoleira. Com saudade da ribeira. Terra do meu bem que...",Péricles
3914,Coração da Massa,"Laiá, Laiá, Laiá, Laiá lá. Laiá, Laiá, Laiá lá. Laiá, Laiá, Laiá. E quem sou eu. Você sabe quem ...",Fundo de Quintal
3183,Medley: Pra Amenizar Teu Coração - na Boutique - Pra Ninguém Mais Chorar,Te dei amor. Te dei um mar de emoção. Em troca levas-te o meu coração. A minha viola e um samba ...,Diogo Nogueira


In [37]:
pd.options.display.max_colwidth = 100
letras_ag.query('grupo == 2')[["SName", "Lyric", "Artist"]].sample(10)

,SName,Lyric,Artist
4731,A Brincadeira do Mundo,"Na brincadeira do mundo eu não podia brincar,. Porque o mundo parou, minha tristeza ficou,. E to...",Jair Rodrigues
6070,Rumo Ao Infinito,Tenho tanto pra falar. Não deixe que o mau tempo venha pra desarrumar. Se a gente arrumou. Ningu...,Maria Rita
2086,O Medo de Ficar Pobre,"Só bato cabeça pra vovó!!. Eu só bato cabeça pra vovó!!. Sim, é pra vovó!!. A cabeça que eu bato...",Bezerra da Silva
2735,Um Tempo Que Passou,"Vou. Uma vez mais. Correr atrás. De todo o meu tempo perdido. Quem sabe, está guardado. Num reló...",Chico Buarque
1103,"Vida, Vida",Embarcou pra vida. Nas asas da primavera. A paixão saída. Do colo da quimera. No olhar travesso....,Almir Guineto
1863,Se Vira,"Então malandragem. Agora você, se vira. Se liga no fato, rasguei o contrato. Acabou a mentira. A...",Beth Carvalho
6889,Deixa pra Amanhã,Roupas espalhadas pelo quarto. O dia amanheceu e eu nem senti. Talvez eu falte hoje ao trabalho....,Péricles
5235,Cachê E Couvert,"Sonhar é o meu jeito. De querer, só um modo. De gostar, sagrado meu. Segredo, de tanto meu. Teu ...",Jorge Aragão
965,Necessidade,"Beijar tua boca é a minha aventura,. Voar pelos ventos do seu bem-querer,. Beijar tua boca é per...",Alexandre Pires
1104,Vovó Rezadeira,Já estou de bem com a vida. A tristeza foi embora. Minhas lágrimas sofridas. Já botei elas pra f...,Almir Guineto


Artista mais presentes nesse grupo

In [40]:
for g in range(0, 7):
    print("\n-----\nGrupo {}:".format(g))
    print(letras_ag.query('grupo == {}'.format(g))["Artist"].value_counts()[0:10])
    print("-----")


-----
Grupo 0:
Raça Negra         65
Alcione            53
Belo               47
Rodriguinho        45
Pixote             43
ExaltaSamba        42
Turma do Pagode    27
Thiaguinho         26
Alexandre Pires    23
Cartola            23
Name: Artist, dtype: int64
-----

-----
Grupo 1:
Bezerra da Silva     262
Chico Buarque        229
Jorge Ben Jor        193
Zeca Pagodinho       140
Marcelo D2           130
Alcione              113
Martinho da Vila     111
Harmonia do Samba    102
Maria Rita           100
Clara Nunes           98
Name: Artist, dtype: int64
-----

-----
Grupo 2:
Alcione             71
Maria Rita          64
Belo                59
Chico Buarque       58
Turma do Pagode     52
Fundo de Quintal    48
Zeca Pagodinho      44
Raça Negra          44
Marcelo D2          42
Jorge Aragão        41
Name: Artist, dtype: int64
-----

-----
Grupo 3:
Alcione             46
Raça Negra          44
Turma do Pagode     36
Thiaguinho          35
ExaltaSamba         32
Fundo de Quintal    30

In [41]:
letras_ag.query('Artist == "Raça Negra"')[['grupo', 'SName']].groupby("grupo").count()

,SName
grupo,
0,65
1,24
2,44
3,44
4,83


Visualizando grupos em muitas dimensões

Para visualizar melhor estruturas de semelhanças nos nossos dados, precisamos que eles estejam em 2D, usaremos um método de **redução de dimensionalidade**.

Os dois métodos mais frequentemente usados são PCA e t-SNE.


In [43]:
from sklearn.manifold import TSNE

letras_embedded = TSNE(n_components=2, verbose=1, perplexity=45).fit_transform(vec_text)

c:\Users\gigi\AppData\Local\Programs\Python\Python310\lib\site-packages\sklearn\manifold\_t_sne.py:800: FutureWarning: The default initialization in TSNE will change from 'random' to 'pca' in 1.2.
  warnings.warn(
c:\Users\gigi\AppData\Local\Programs\Python\Python310\lib\site-packages\sklearn\manifold\_t_sne.py:810: FutureWarning: The default learning rate in TSNE will change from 200.0 to 'auto' in 1.2.
  warnings.warn(


[t-SNE] Computing 136 nearest neighbors...
[t-SNE] Indexed 8662 samples in 0.009s...
[t-SNE] Computed neighbors for 8662 samples in 6.028s...
[t-SNE] Computed conditional probabilities for sample 1000 / 8662
[t-SNE] Computed conditional probabilities for sample 2000 / 8662
[t-SNE] Computed conditional probabilities for sample 3000 / 8662
[t-SNE] Computed conditional probabilities for sample 4000 / 8662
[t-SNE] Computed conditional probabilities for sample 5000 / 8662
[t-SNE] Computed conditional probabilities for sample 6000 / 8662
[t-SNE] Computed conditional probabilities for sample 7000 / 8662
[t-SNE] Computed conditional probabilities for sample 8000 / 8662
[t-SNE] Computed conditional probabilities for sample 8662 / 8662
[t-SNE] Mean sigma: 0.297724
[t-SNE] KL divergence after 50 iterations with early exaggeration: 90.441841
[t-SNE] KL divergence after 1000 iterations: 2.996449


In [44]:
letras_ag = letras_ag.assign(tsnel = letras_embedded[:,0], tsne2 = letras_embedded[:,1])

alt.Chart(letras_ag.sample(500)).mark_circle(
    opacity = .7,
    size = 30
).encode(
    x = "tsnel",
    y = "tsne2",
    color = "grupo:N",
    tooltip = ["Artist", "SName"]
).interactive()

c:\Users\gigi\AppData\Local\Programs\Python\Python310\lib\site-packages\altair\utils\core.py:317: FutureWarning: iteritems is deprecated and will be removed in a future version. Use .items instead.
  for col_name, dtype in df.dtypes.iteritems():


alt.Chart(...)